In [9]:
# SGD- Random Forest:

#All imports declarations
import pandas as pd
import numpy as np
import csv

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1)     

def add_context(data_df, context = 0):
    L = len(data_df)
    cols = list(data_df.columns)
    context_data = []
    row_1 = np.asarray(data_df.iloc[0,:])
    row_1 = np.reshape(row_1,(1,-1))
    row_1 = pd.DataFrame(row_1, columns=cols)
    #pad data with data[0] context times
    data = pd.concat([row_1]*context + [data_df])
    try:
        del data[0]
    except:
        pass
    
    data = data.to_numpy()
    for fi in range(context, len(data)):
        frame = data[fi-context:fi+1].flatten()
        context_data.append(frame)
    return pd.DataFrame(context_data, columns = cols*(context+1))

'''def read_dataset(filename):
    dataset = pd.read_csv(filename)
    df_cols = list(dataset.columns)
    dataset = scaler.fit_transform(dataset)
    dataset = pd.DataFrame(dataset, columns = df_cols)
    return dataset'''

'''
updated read_dataset Function
'''    
def read_dataset(filename):
    dataset = pd.read_csv(filename)
    return dataset

def get_data_labels(df, context = 0):
    label = df['flow_size']
    #label = df['flow_size']
    try:
      del df['job']
    except:
      pass
    df = add_context(df, context)
    df.iloc[:, -1] = 0
    #add context to labels
    return df, label


def plot_features(labels,values,title = 'Graph',rot = 90):
    index = np.arange(len(labels))
    plt.bar(index*2.0,values,width = 1)
    plt.xticks(index*2.0, labels, fontsize=15, rotation=rot)
    plt.title(title)
    plt.show()
'''
Updated train, test values '''
#need to scale values to the range(0,1)
scaler = MinMaxScaler(feature_range=(0,1))

train_df = read_dataset('SGD_training.csv')
test_df = read_dataset('SGD_test.csv')
validation_df= read_dataset('SGD_validation.csv')

#scale and convert to df
#fit on train, transform train, test, validation
cols = list(train_df.columns)
train_df = scaler.fit_transform(train_df)
train_df = pd.DataFrame(train_df, columns = cols)

test_df = scaler.transform(test_df)
test_df = pd.DataFrame(test_df, columns = cols)

validation_df = scaler.transform(validation_df)
validation_df = pd.DataFrame(validation_df, columns = cols)

context = 1

train_x, train_y = get_data_labels(train_df, context)
test_x, test_y = get_data_labels(test_df, context)
validation_x, validation_y = get_data_labels(validation_df, context)

'''for i in range(1,20):
  rfc = RandomForestRegressor(n_estimators=i)
  rfc.fit(train_x, train_y)
  predict_y = rfc.predict(test_x)
  print("size testdf:", len(test_df))
  print("size traindf:", len(train_df))
  print("size test_x:", len(test_x))
  print("size train_x:", len(train_x))
  print(i, r2_score(test_y, predict_y))'''



1 0.1677480204560602
2 -0.10969860722330771
3 0.058444425368974984
4 0.27632827475067234
5 0.13125661712897685
6 0.1725854816541278
7 0.21841283316938298
8 0.3312008362389025
9 0.388524756909286
10 0.3154979811279093
11 0.1761524922299651
12 0.222047643597877
13 0.346536198505659
14 0.31892956594922084
15 0.16873609997574246
16 0.09460996621746864
17 0.2902495819247325
18 0.42163978297815685
19 0.30777756429841296


In [11]:
rfc = RandomForestRegressor(n_estimators=19)
rfc.fit(train_x, train_y)
predict_y = rfc.predict(test_x)
print(19, r2_score(test_y, predict_y))

19 0.3823405880412115


In [12]:
test_mini_df = read_dataset('SGD_test_jb_mini.csv')
#jobs = test_mini_df['job']
#del test_mini_df['job']
print('predict_y:', len(predict_y))
print('test_df',test_df)
#get original test_df
test_df['flow_size'] = predict_y
#print(test_df.tail())
unscaled_test_df = scaler.inverse_transform(test_df)
unscaled_test_df = pd.DataFrame(unscaled_test_df, columns = cols)
#change_cols = {"agg_net_out":'int32', "agg_net_in":'int32', "agg_net_out_per_machine":'int32','agg_net_in_per_machine':'int32','machine':'int32','flow_size':'int32'}
#unscaled_test_df = unscaled_test_df.astype(change_cols)
unscaled_test_df.tail(20)

new_df = pd.DataFrame([])
new_df['flow_size'] = test_mini_df['flow_size']
new_df['job'] = test_mini_df['job']
new_df['predicted_flow_size'] = test_mini_df['flow_size']

#now copy predicted
for index, row in test_mini_df.iterrows():
  i = int(row['pseudo_index'])
  new_df.iloc[index,2] = unscaled_test_df.iloc[i]['flow_size']

new_df.to_csv('SGD_test_jb_mini_predicted_no_outliers.csv')

predict_y: 57531
test_df            time  first_call  machine  ...  agg_net_in  flow_frequency  flow_size
0      0.999448    0.000919  0.00000  ...    0.000001    0.000000e+00   0.000002
1      0.999448    0.000689  0.00000  ...    0.000002    6.626595e-09   0.000001
2      0.999448    0.000689  0.00000  ...    0.000002    1.744291e-07   0.000001
3      0.999448    0.000689  0.00000  ...    0.000003    1.728982e-07   0.000001
4      0.999448    0.000689  0.00000  ...    0.000004    1.728667e-07   0.000001
...         ...         ...      ...  ...         ...             ...        ...
57526  0.999617    0.000832  0.00000  ...    0.796774    0.000000e+00   0.000001
57527  0.999617    0.000716  0.00641  ...    0.796774    4.252106e-08   0.000001
57528  0.999617    0.000832  0.00000  ...    0.796774    1.302607e-07   0.000001
57529  0.999618    0.000832  0.00000  ...    0.796775    1.728106e-07   0.000001
57530  0.999618    0.000832  0.00000  ...    0.796778    1.727876e-07   0.000001

[5

In [17]:
# KMeans- Random Forest:

#All imports declarations
import pandas as pd
import numpy as np
import csv

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1)     

def add_context(data_df, context = 0):
    L = len(data_df)
    cols = list(data_df.columns)
    context_data = []
    row_1 = np.asarray(data_df.iloc[0,:])
    row_1 = np.reshape(row_1,(1,-1))
    row_1 = pd.DataFrame(row_1, columns=cols)
    #pad data with data[0] context times
    data = pd.concat([row_1]*context + [data_df])
    try:
        del data[0]
    except:
        pass
    
    data = data.to_numpy()
    for fi in range(context, len(data)):
        frame = data[fi-context:fi+1].flatten()
        context_data.append(frame)
    return pd.DataFrame(context_data, columns = cols*(context+1))

'''def read_dataset(filename):
    dataset = pd.read_csv(filename)
    df_cols = list(dataset.columns)
    dataset = scaler.fit_transform(dataset)
    dataset = pd.DataFrame(dataset, columns = df_cols)
    return dataset'''

'''
updated read_dataset Function
'''    
def read_dataset(filename):
    dataset = pd.read_csv(filename)
    return dataset

def get_data_labels(df, context = 0):
    label = df['flow_size']
    #label = df['flow_size']
    try:
      del df['job']
    except:
      pass
    df = add_context(df, context)
    df.iloc[:, -1] = 0
    #add context to labels
    return df, label


def plot_features(labels,values,title = 'Graph',rot = 90):
    index = np.arange(len(labels))
    plt.bar(index*2.0,values,width = 1)
    plt.xticks(index*2.0, labels, fontsize=15, rotation=rot)
    plt.title(title)
    plt.show()
'''
Updated train, test values '''
#need to scale values to the range(0,1)
scaler = MinMaxScaler(feature_range=(0,1))

train_df = read_dataset('KMeans_training.csv')
test_df = read_dataset('KMeans_test.csv')
validation_df= read_dataset('KMeans_validation.csv')

#scale and convert to df
#fit on train, transform train, test, validation
cols = list(train_df.columns)
train_df = scaler.fit_transform(train_df)
train_df = pd.DataFrame(train_df, columns = cols)

test_df = scaler.transform(test_df)
test_df = pd.DataFrame(test_df, columns = cols)

validation_df = scaler.transform(validation_df)
validation_df = pd.DataFrame(validation_df, columns = cols)

context = 1

train_x, train_y = get_data_labels(train_df, context)
test_x, test_y = get_data_labels(test_df, context)
validation_x, validation_y = get_data_labels(validation_df, context)

for i in range(1,20):
  rfc = RandomForestRegressor(n_estimators=i)
  rfc.fit(train_x, train_y)
  predict_y = rfc.predict(test_x)
  #print("size testdf:", len(test_df))
  #print("size traindf:", len(train_df))
  #print("size test_x:", len(test_x))
  #print("size train_x:", len(train_x))
  print(i, r2_score(test_y, predict_y))



1 0.8193058378072318
2 0.893250688327546
3 0.8484804431476178
4 0.8690003529666837
5 0.8789231554427571
6 0.8946355322412317
7 0.8745326929628829
8 0.8909622797031816
9 0.8958775948556043
10 0.899345826667517
11 0.8984087445904212
12 0.8958658072145758
13 0.8908794054592907
14 0.8803767426835489
15 0.8973523046772676
16 0.8980961748829122
17 0.8931272365177727
18 0.9050718577854955
19 0.9059467778204662


In [18]:
rfc = RandomForestRegressor(n_estimators=19)
rfc.fit(train_x, train_y)
predict_y = rfc.predict(test_x)
print(19, r2_score(test_y, predict_y))

19 0.898272731062778


In [21]:
test_mini_df = read_dataset('Kmeans_test_jb_mini.csv')
#jobs = test_mini_df['job']
#del test_mini_df['job']
print('predict_y:', len(predict_y))
print('test_df',test_df)
#get original test_df
test_df['flow_size'] = predict_y
#print(test_df.tail())
unscaled_test_df = scaler.inverse_transform(test_df)
unscaled_test_df = pd.DataFrame(unscaled_test_df, columns = cols)
#change_cols = {"agg_net_out":'int32', "agg_net_in":'int32', "agg_net_out_per_machine":'int32','agg_net_in_per_machine':'int32','machine':'int32','flow_size':'int32'}
#unscaled_test_df = unscaled_test_df.astype(change_cols)
unscaled_test_df.tail(20)

new_df = pd.DataFrame([])
new_df['flow_size'] = test_mini_df['flow_size']
new_df['job'] = test_mini_df['job']
new_df['predicted_flow_size'] = test_mini_df['flow_size']

#now copy predicted
for index, row in test_mini_df.iterrows():
  i = int(row['pseudo_index'])
  new_df.iloc[index,2] = unscaled_test_df.iloc[i]['flow_size']

new_df.to_csv('KMeans_test_jb_mini_predicted_no_outliers.csv')

predict_y: 15730
test_df            time  previous_flow_time  ...  first_call     flow_size
0      0.060262            0.040180  ...    0.015248  2.508981e-06
1      0.078166            0.019965  ...    0.026744  4.400595e-06
2      0.080349            0.001248  ...    0.031606  5.200703e-06
3      0.082314            0.001497  ...    0.015528  2.555078e-06
4      0.084061            0.001497  ...    0.014428  2.373983e-06
...         ...                 ...  ...         ...           ...
15725  0.837773            0.325430  ...    0.079221  1.303551e-05
15726  0.837773            0.325430  ...    0.000520  8.560828e-08
15727  0.837773            0.009483  ...    0.000000  0.000000e+00
15728  0.837773            0.325680  ...    0.000000  0.000000e+00
15729  0.838646            0.014724  ...    0.000520  8.560828e-08

[15730 rows x 15 columns]


In [22]:
# PageRank- Random Forest:

#All imports declarations
import pandas as pd
import numpy as np
import csv

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1)     

def add_context(data_df, context = 0):
    L = len(data_df)
    cols = list(data_df.columns)
    context_data = []
    row_1 = np.asarray(data_df.iloc[0,:])
    row_1 = np.reshape(row_1,(1,-1))
    row_1 = pd.DataFrame(row_1, columns=cols)
    #pad data with data[0] context times
    data = pd.concat([row_1]*context + [data_df])
    try:
        del data[0]
    except:
        pass
    
    data = data.to_numpy()
    for fi in range(context, len(data)):
        frame = data[fi-context:fi+1].flatten()
        context_data.append(frame)
    return pd.DataFrame(context_data, columns = cols*(context+1))

'''def read_dataset(filename):
    dataset = pd.read_csv(filename)
    df_cols = list(dataset.columns)
    dataset = scaler.fit_transform(dataset)
    dataset = pd.DataFrame(dataset, columns = df_cols)
    return dataset'''

'''
updated read_dataset Function
'''    
def read_dataset(filename):
    dataset = pd.read_csv(filename)
    return dataset

def get_data_labels(df, context = 0):
    label = df['flow_size']
    #label = df['flow_size']
    try:
      del df['job']
    except:
      pass
    df = add_context(df, context)
    df.iloc[:, -1] = 0
    #add context to labels
    return df, label


def plot_features(labels,values,title = 'Graph',rot = 90):
    index = np.arange(len(labels))
    plt.bar(index*2.0,values,width = 1)
    plt.xticks(index*2.0, labels, fontsize=15, rotation=rot)
    plt.title(title)
    plt.show()
'''
Updated train, test values '''
#need to scale values to the range(0,1)
scaler = MinMaxScaler(feature_range=(0,1))

train_df = read_dataset('PageRank_training.csv')
test_df = read_dataset('PageRank_test.csv')
validation_df= read_dataset('PageRank_validation.csv')

#scale and convert to df
#fit on train, transform train, test, validation
cols = list(train_df.columns)
train_df = scaler.fit_transform(train_df)
train_df = pd.DataFrame(train_df, columns = cols)

test_df = scaler.transform(test_df)
test_df = pd.DataFrame(test_df, columns = cols)

validation_df = scaler.transform(validation_df)
validation_df = pd.DataFrame(validation_df, columns = cols)

context = 1

train_x, train_y = get_data_labels(train_df, context)
test_x, test_y = get_data_labels(test_df, context)
validation_x, validation_y = get_data_labels(validation_df, context)

for i in range(1,20):
  rfc = RandomForestRegressor(n_estimators=i)
  rfc.fit(train_x, train_y)
  predict_y = rfc.predict(test_x)
  #print("size testdf:", len(test_df))
  #print("size traindf:", len(train_df))
  #print("size test_x:", len(test_x))
  #print("size train_x:", len(train_x))
  print(i, r2_score(test_y, predict_y))



1 0.6983204319894609
2 0.7101900756688395
3 0.757167437911398
4 0.8269124666741674
5 0.7651784475131902
6 0.7824328319791763
7 0.7721773807395867
8 0.7975466126788959
9 0.8290804971044602
10 0.8476087709866041
11 0.8267697655301125
12 0.8231675467612243
13 0.7711488647507884
14 0.8211419930522503
15 0.8527223829500807
16 0.8526860553836406
17 0.8310418960705626
18 0.8142875091101824
19 0.8370081084500898


In [23]:
rfc = RandomForestRegressor(n_estimators=15)
rfc.fit(train_x, train_y)
predict_y = rfc.predict(test_x)
print(15, r2_score(test_y, predict_y))

15 0.8399089953846118


In [24]:
test_mini_df = read_dataset('PageRank_test_jb_mini.csv')
#jobs = test_mini_df['job']
#del test_mini_df['job']
print('predict_y:', len(predict_y))
print('test_df',test_df)
#get original test_df
test_df['flow_size'] = predict_y
#print(test_df.tail())
unscaled_test_df = scaler.inverse_transform(test_df)
unscaled_test_df = pd.DataFrame(unscaled_test_df, columns = cols)
#change_cols = {"agg_net_out":'int32', "agg_net_in":'int32', "agg_net_out_per_machine":'int32','agg_net_in_per_machine':'int32','machine':'int32','flow_size':'int32'}
#unscaled_test_df = unscaled_test_df.astype(change_cols)
unscaled_test_df.tail(20)

new_df = pd.DataFrame([])
new_df['flow_size'] = test_mini_df['flow_size']
new_df['job'] = test_mini_df['job']
new_df['predicted_flow_size'] = test_mini_df['flow_size']

#now copy predicted
for index, row in test_mini_df.iterrows():
  i = int(row['pseudo_index'])
  new_df.iloc[index,2] = unscaled_test_df.iloc[i]['flow_size']

new_df.to_csv('PageRank_test_jb_mini_predicted_no_outliers.csv')

predict_y: 14814
test_df            time  flow_frequency  ...   machine     flow_size
0      0.060141        0.025600  ...  0.666667  9.449852e-07
1      0.067399        0.009333  ...  0.666667  2.922210e-06
2      0.068644        0.000800  ...  0.666667  6.191793e-06
3      0.069681        0.001333  ...  0.666667  8.099873e-07
4      0.070925        0.000533  ...  0.666667  4.892027e-05
...         ...             ...  ...       ...           ...
14809  0.469307        0.002667  ...  0.666667  4.581697e-05
14810  0.469929        0.000800  ...  0.666667  3.121908e-05
14811  0.471174        0.001333  ...  0.666667  3.139030e-05
14812  0.471796        0.204800  ...  0.000000  1.768966e-05
14813  0.000000        0.000000  ...  0.000000  2.825078e-06

[14814 rows x 8 columns]


In [25]:
#GradientBoosted Decision tree: PageRank:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1)     


def add_context(data_df, context = 0):

    L = len(data_df)
    cols = list(data_df.columns)

    context_data = []
    
    row_1 = np.asarray(data_df.iloc[0,:])
    row_1 = np.reshape(row_1,(1,-1))
    row_1 = pd.DataFrame(row_1, columns=cols)
    #pad data with data[0] context times
    data = pd.concat([row_1]*context + [data_df])
    
    try:
        del data[0]
    except:
        pass
    
    data = data.to_numpy()
    
    
    for fi in range(context, len(data)):
        frame = data[fi-context:fi+1].flatten()
        context_data.append(frame)

    return pd.DataFrame(context_data, columns = cols*(context+1))

'''def read_dataset(filename):
    dataset = pd.read_csv(filename)
    df_cols = list(dataset.columns)
    #dataset = scaler.fit_transform(dataset)
    dataset = pd.DataFrame(dataset, columns = df_cols)
    return dataset
'''

def read_dataset(filename):
    dataset = pd.read_csv(filename)
    return dataset

def get_data_labels(df, context = 0):
    label = df['flow_size']
    try:
      del df['job']
    except:
      pass
    df = add_context(df, context)
    df.iloc[:, -1] = 0
    #add context to labels
    return df, label



scaler = MinMaxScaler(feature_range=(0, 1))

train_df = read_dataset('PageRank_training.csv')
test_df = read_dataset('PageRank_test.csv')

#context = 4
context=1

cols = train_df.columns

#scale and convert to df
#fit on train, transform train, test, validation
train_df = scaler.fit_transform(train_df)
train_df = pd.DataFrame(train_df, columns = cols)

test_df = scaler.transform(test_df)
test_df = pd.DataFrame(test_df, columns = cols)


training_x, training_y = get_data_labels(train_df, context)
testing_x, testing_y = get_data_labels(test_df, context)


regItem = GradientBoostingRegressor( max_depth=10, n_estimators=50, learning_rate=1.0 )
regItem.fit(training_x, training_y)
list_of_err=[]

for predicted_y in regItem.staged_predict(testing_x):
    list_of_err.append(mean_squared_error(testing_y, predicted_y))
topEst = np.argmin(list_of_err)

bestRegItem = GradientBoostingRegressor( max_depth=7, n_estimators=topEst+1, learning_rate=1.0 )
bestRegItem.fit(training_x, training_y)
#predicted_score = bestRegItem.score(testing_x,testing_y)
#predicted_y = bestRegItem.predict(testing_x)

predicted_score = bestRegItem.score(testing_x,testing_y)
predicted_y = bestRegItem.predict(testing_x)

print("Accuracy is ",predicted_score)
print("R2 is ", r2_score(testing_y, predicted_y))



Accuracy is  0.5986977910437798
R2 is  0.5986977910437798


In [0]:
test_mini_df = read_dataset('PageRank_test_jb_mini.csv')
#jobs = test_mini_df['job']
#del test_mini_df['job']

#get original test_df
test_df['flow_size'] = predicted_y
#print(test_df.tail())
unscaled_test_df = scaler.inverse_transform(test_df)
unscaled_test_df = pd.DataFrame(unscaled_test_df, columns = cols)
#change_cols = {"agg_net_out":'int32', "agg_net_in":'int32', "agg_net_out_per_machine":'int32','agg_net_in_per_machine':'int32','machine':'int32','flow_size':'int32'}
#unscaled_test_df = unscaled_test_df.astype(change_cols)
unscaled_test_df.tail(20)

new_df = pd.DataFrame([])
new_df['flow_size'] = test_mini_df['flow_size']
new_df['job'] = test_mini_df['job']
new_df['predicted_flow_size'] = test_mini_df['flow_size']

#now copy predicted
for index, row in test_mini_df.iterrows():
  i = int(row['pseudo_index'])
  new_df.iloc[index,2] = unscaled_test_df.iloc[i]['flow_size']

new_df.to_csv('PageRank_test_jb_mini_predicted_no_outliers_GBDT.csv')



In [28]:
#GradientBoosted Decision tree: KMeans:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1)     


def add_context(data_df, context = 0):

    L = len(data_df)
    cols = list(data_df.columns)

    context_data = []
    
    row_1 = np.asarray(data_df.iloc[0,:])
    row_1 = np.reshape(row_1,(1,-1))
    row_1 = pd.DataFrame(row_1, columns=cols)
    #pad data with data[0] context times
    data = pd.concat([row_1]*context + [data_df])
    
    try:
        del data[0]
    except:
        pass
    
    data = data.to_numpy()
    
    
    for fi in range(context, len(data)):
        frame = data[fi-context:fi+1].flatten()
        context_data.append(frame)

    return pd.DataFrame(context_data, columns = cols*(context+1))

def read_dataset(filename):
    dataset = pd.read_csv(filename)
    return dataset

def get_data_labels(df, context = 0):
    label = df['flow_size']
    try:
      del df['job']
    except:
      pass
    df = add_context(df, context)
    df.iloc[:, -1] = 0
    #add context to labels
    return df, label



scaler = MinMaxScaler(feature_range=(0, 1))

train_df = read_dataset('KMeans_training.csv')
test_df = read_dataset('KMeans_test.csv')

#context = 4
context=1

cols = train_df.columns

#scale and convert to df
#fit on train, transform train, test, validation
train_df = scaler.fit_transform(train_df)
train_df = pd.DataFrame(train_df, columns = cols)

test_df = scaler.transform(test_df)
test_df = pd.DataFrame(test_df, columns = cols)


training_x, training_y = get_data_labels(train_df, context)
testing_x, testing_y = get_data_labels(test_df, context)


regItem = GradientBoostingRegressor( max_depth=10, n_estimators=50, learning_rate=1.0 )
regItem.fit(training_x, training_y)
list_of_err=[]

for predicted_y in regItem.staged_predict(testing_x):
    list_of_err.append(mean_squared_error(testing_y, predicted_y))
topEst = np.argmin(list_of_err)

bestRegItem = GradientBoostingRegressor( max_depth=7, n_estimators=topEst+1, learning_rate=1.0 )
bestRegItem.fit(training_x, training_y)
#predicted_score = bestRegItem.score(testing_x,testing_y)
#predicted_y = bestRegItem.predict(testing_x)

predicted_score = bestRegItem.score(testing_x,testing_y)
predicted_y = bestRegItem.predict(testing_x)

print("Accuracy is ",predicted_score)
print("R2 is ", r2_score(testing_y, predicted_y))



Accuracy is  0.8548974600609157
R2 is  0.8548974600609157


In [0]:
test_mini_df = read_dataset('Kmeans_test_jb_mini.csv')
#jobs = test_mini_df['job']
#del test_mini_df['job']

#get original test_df
test_df['flow_size'] = predicted_y
#print(test_df.tail())
unscaled_test_df = scaler.inverse_transform(test_df)
unscaled_test_df = pd.DataFrame(unscaled_test_df, columns = cols)
#change_cols = {"agg_net_out":'int32', "agg_net_in":'int32', "agg_net_out_per_machine":'int32','agg_net_in_per_machine':'int32','machine':'int32','flow_size':'int32'}
#unscaled_test_df = unscaled_test_df.astype(change_cols)
unscaled_test_df.tail(20)

new_df = pd.DataFrame([])
new_df['flow_size'] = test_mini_df['flow_size']
new_df['job'] = test_mini_df['job']
new_df['predicted_flow_size'] = test_mini_df['flow_size']

#now copy predicted
for index, row in test_mini_df.iterrows():
  i = int(row['pseudo_index'])
  new_df.iloc[index,2] = unscaled_test_df.iloc[i]['flow_size']

new_df.to_csv('KMeans_test_jb_mini_predicted_no_outliers_GBDT.csv')



In [30]:
#GradientBoosted Decision tree: SGD:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1)     


def add_context(data_df, context = 0):

    L = len(data_df)
    cols = list(data_df.columns)

    context_data = []
    
    row_1 = np.asarray(data_df.iloc[0,:])
    row_1 = np.reshape(row_1,(1,-1))
    row_1 = pd.DataFrame(row_1, columns=cols)
    #pad data with data[0] context times
    data = pd.concat([row_1]*context + [data_df])
    
    try:
        del data[0]
    except:
        pass
    
    data = data.to_numpy()
    
    
    for fi in range(context, len(data)):
        frame = data[fi-context:fi+1].flatten()
        context_data.append(frame)

    return pd.DataFrame(context_data, columns = cols*(context+1))

def read_dataset(filename):
    dataset = pd.read_csv(filename)
    return dataset

def get_data_labels(df, context = 0):
    label = df['flow_size']
    try:
      del df['job']
    except:
      pass
    df = add_context(df, context)
    df.iloc[:, -1] = 0
    #add context to labels
    return df, label



scaler = MinMaxScaler(feature_range=(0, 1))

train_df = read_dataset('SGD_training.csv')
test_df = read_dataset('SGD_test.csv')

#context = 4
context=1

cols = train_df.columns

#scale and convert to df
#fit on train, transform train, test, validation
train_df = scaler.fit_transform(train_df)
train_df = pd.DataFrame(train_df, columns = cols)

test_df = scaler.transform(test_df)
test_df = pd.DataFrame(test_df, columns = cols)


training_x, training_y = get_data_labels(train_df, context)
testing_x, testing_y = get_data_labels(test_df, context)


regItem = GradientBoostingRegressor( max_depth=10, n_estimators=50, learning_rate=1.0 )
regItem.fit(training_x, training_y)
list_of_err=[]

for predicted_y in regItem.staged_predict(testing_x):
    list_of_err.append(mean_squared_error(testing_y, predicted_y))
topEst = np.argmin(list_of_err)

bestRegItem = GradientBoostingRegressor( max_depth=7, n_estimators=topEst+1, learning_rate=1.0 )
bestRegItem.fit(training_x, training_y)
#predicted_score = bestRegItem.score(testing_x,testing_y)
#predicted_y = bestRegItem.predict(testing_x)

predicted_score = bestRegItem.score(testing_x,testing_y)
predicted_y = bestRegItem.predict(testing_x)

print("Accuracy is ",predicted_score)
print("R2 is ", r2_score(testing_y, predicted_y))



Accuracy is  0.6110783307902424
R2 is  0.6110783307902424


In [0]:
test_mini_df = read_dataset('SGD_test_jb_mini.csv')
#jobs = test_mini_df['job']
#del test_mini_df['job']

#get original test_df
test_df['flow_size'] = predicted_y
#print(test_df.tail())
unscaled_test_df = scaler.inverse_transform(test_df)
unscaled_test_df = pd.DataFrame(unscaled_test_df, columns = cols)
#change_cols = {"agg_net_out":'int32', "agg_net_in":'int32', "agg_net_out_per_machine":'int32','agg_net_in_per_machine':'int32','machine':'int32','flow_size':'int32'}
#unscaled_test_df = unscaled_test_df.astype(change_cols)
unscaled_test_df.tail(20)

new_df = pd.DataFrame([])
new_df['flow_size'] = test_mini_df['flow_size']
new_df['job'] = test_mini_df['job']
new_df['predicted_flow_size'] = test_mini_df['flow_size']

#now copy predicted
for index, row in test_mini_df.iterrows():
  i = int(row['pseudo_index'])
  new_df.iloc[index,2] = unscaled_test_df.iloc[i]['flow_size']

new_df.to_csv('SGD_test_jb_mini_predicted_no_outliers_GBDT.csv')

